# Preprocessing of the dataset  

## Steps :
* Merging the differents datasets into one
* Creating a date column
* Removing column with 70% of null values
* Creating numerical value for the full time result

### Merging the datasets
Merge datasets from season 14-15 to 19-20. Season 20-21 will be kept as testing data so we leave it alone for the moment. 

In [11]:
import os 
import pandas as pd


path = "../assets/data/"
def merged_df():
    
    initial_file = "PL_14-15.csv"
    initial_df = pd.read_csv(path+initial_file)
    files_to_append = [f for f in os.listdir(path) if f != initial_file]
    for f in files_to_append:
        df = pd.read_csv(path+f)
        initial_df = initial_df.append(df)
    return initial_df
df = merged_df()

In [29]:
import numpy as np

## Creating a date column

In [12]:
df["Date"] = pd.to_datetime(df["Date"])

In [13]:
df.isnull().mean() *100

Div               0.021925
Date              0.021925
HomeTeam          0.021925
AwayTeam          0.021925
FTHG              0.021925
                   ...    
MaxCAHA          91.668494
AvgCAHH          91.668494
AvgCAHA          91.668494
Numerical_htr    50.010963
Numerical_ftr    50.010963
Length: 132, dtype: float64

## Removing rows with missing values in critical columns
We cant afford to have missing values inside of FTR, HomeTeam, AwayTeam columns. We remove the rows with missing values at those points

In [14]:
df.dropna(subset=["FTR", "HomeTeam", "AwayTeam"], inplace=True, how="any")

## Removing column with 70% of null values

In [15]:
columns_to_drop = [col for col in df.columns if df[col].isnull().mean() *100 > 75 ]
df.drop(columns_to_drop, axis=1, inplace= True)

## Removing odds column with too much null values


Some odds column have more null values than other. The odds are very important in the model and just replacing with the median or the mean  could have a negative effect on our predicitions. 

In [16]:
odds_columns = df.iloc[:,23:]
odds_columns.isnull().sum()
odds_columns_to_drop = [c for c in odds_columns.columns if odds_columns[c].isnull().sum() >1]
odds_columns.drop(odds_columns_to_drop, axis=1, inplace=True)

## Creating numerical value for the full time result. 
The full time result (FTR) column is a string but in order to process it later we need to encode it as a numerical value. 

In [19]:
def numerize_ftr_htr(ftr):
    if ftr == "D":
        return 0
    elif ftr == "A":
        return 2
    elif ftr == "H":
        return 1
    else :
        return None
df["Numerical_htr"] = df["HTR"].apply(numerize_ftr_htr)
df["Numerical_ftr"] = df["FTR"].apply(numerize_ftr_htr)

In [20]:
df.to_csv('../assets/data/clean_data.csv', index=None)

## Creating average odd for home win/draw/away win

In [32]:
home_odds  = [c for c in odds_columns if c[-1] == "H"]
away_odds = [c for c in odds_columns if c[-1]=="A"]
draw_odds = [c for c in odds_columns if c[-1] =="D"]

In [30]:
odds_columns["home_odds_mean"] = odds_columns[home_odds].apply(np.mean, axis=1)

In [42]:
odds_columns["away_odds_mean"] = odds_columns[away_odds].apply(np.mean, axis=1)

In [43]:
odds_columns["draw_odds_mean"] = odds_columns[draw_odds].apply(np.mean, axis=1)

In [37]:
print(away_odds)
print(draw_odds)
print(home_odds)

['B365A', 'PSA', 'WHA', 'VCA', 'PSCA']
['B365D', 'PSD', 'WHD', 'VCD', 'PSCD']
['B365H', 'PSH', 'WHH', 'VCH', 'PSCH']


In [44]:
odds_columns[["home_odds_mean","away_odds_mean", "draw_odds_mean"]]

,home_odds_mean,away_odds_mean,draw_odds_mean
0,1.260,12.852,6.120
1,3.150,2.426,3.336
2,1.380,9.770,4.922
3,2.488,3.186,3.200
4,1.962,4.502,3.362
...,...,...,...
2275,1.304,10.704,5.782
2276,1.280,10.384,6.328
2277,1.410,8.222,4.898
2278,2.082,3.698,3.590


In [45]:
odds_columns.iloc[0]

B365H              1.250
B365D              6.500
B365A             15.000
PSH                1.260
PSD                6.450
PSA               14.010
WHH                1.250
WHD                5.500
WHA               12.000
VCH                1.250
VCD                6.250
VCA               10.500
PSCH               1.290
PSCD               5.900
PSCA              12.750
home_odds_mean     1.260
away_odds_mean    12.852
draw_odds_mean     6.120
Name: 0, dtype: float64